# Sum 2 or more rasters
With arcpy, this is done with the Cell Statistics function (http://desktop.arcgis.com/en/arcmap/10.3/tools/spatial-analyst-toolbox/cell-statistics.htm) or by summing raster objects (http://desktop.arcgis.com/en/arcmap/10.3/analyze/arcpy-classes/raster-object.htm)

In [15]:
import datetime

## Goal
The goal is to build the function described by the cell below.

In [16]:
def sumRasters(rasters):
    '''
    (list) -> raster object
    
    Sums rasters from a list and returns a raster representing the sum of all layers at each cell.
    
    Arguments:
    rasters -- a python list of rasters (as objects or paths?)
    
    Example:
    >> summed_rasters = sumRasters(["PAD_Protection", "bAMKEx_habitat_map"])
    '''
    summed_raster = ""# summation
    return summed_raster

## Uses
1. Species richness calculation<br>
2. Running habitat models<br>
3. Calculating habitat overlap with areas of interest (e.g., protected areas, risk areas, other areas of interest)<br>


## Environments
Environments affect arcpy function behavior, so it is best and sometimes necessary to define them.

In [17]:
execfile('T:/Scripts/AppendPaths27.py')   # Running this file enables me to import acrpy
import arcpy
arcpy.ResetEnvironments()
arcpy.CheckOutExtension("Spatial")
arcpy.env.overwriteOutput=True
arcpy.env.extent = "MAXOF"
arcpy.env.pyramid = 'PYRAMIDS'
arcpy.env.snapRaster = "" # We use a grid available in here: https://github.com/nmtarr/GAPProduction/tree/master/data
arcpy.env.rasterStatistics = "STATISTICS"
arcpy.env.cellSize = 30

## Locations of data

In [18]:
workDir = "T:/Temp/OpenSourceArcpy/"
dataDir = "P:/Proj3/USGap/Vert/Model/Output/CONUS/TestExtents/"
tinyDir = dataDir + "Tiny/"
mediumDir = dataDir + "Medium/"
smallDir = dataDir + "Small/"

## Example 1 - Sum rasters with Cell Statistics

Step 1 - List the rasters to sum

In [19]:
arcpy.env.workspace = tinyDir
rasters = arcpy.ListRasters()

[u'agptox.tif', u'bamkex.tif', u'batflx.tif', u'bbekix.tif', u'bbethx.tif', u'bbevix.tif', u'bbewrx.tif', u'bbggnx.tif', u'bblphx.tif', u'bbrblx.tif', u'bbtgnx.tif', u'bbtspx.tif', u'bbtywx.tif', u'bbuffx.tif', u'bbuowh.tif', u'bbuowx.tif', u'bcacwx.tif', u'bcanwx.tif', u'bcogdx.tif', u'bcohux.tif', u'bcopox.tif', u'bcorax.tif', u'bewcsx.tif', u'bfehax.tif', u'bgaqux.tif', u'bghowx.tif', u'bgoeax.tif', u'bgrrox.tif', u'bholax.tif', u'bkillx.tif', u'blbwox.tif', u'blcthx.tif', u'blenix.tif', u'bleowx.tif', u'blispx.tif', u'bloshx.tif', u'bmerlx.tif', u'bmodox.tif', u'bnohax.tif', u'bnomox.tif', u'bnrwsx.tif', u'bnshrx.tif', u'bpefax.tif', u'bprfax.tif', u'brckix.tif', u'brlhax.tif', u'browrx.tif', u'brthax.tif', u'bsagsa.tif', u'bsagsn.tif', u'bsagsx.tif', u'bsaphx.tif', u'bseowx.tif', u'btuvux.tif', u'bverdx.tif', u'bvgswx.tif', u'bwekix.tif', u'bwtswx.tif', u'byrwax.tif', u'mabebx.tif', u'mambax.tif', u'mapmox.tif', u'mazwox.tif', u'mbbbax.tif', u'mbftbx.tif', u'mbobcx.tif', u'mbogox.

Step 2 - Run cell statistics <br>NOTE the issue of "nodata" !!!

In [20]:
time1 = datetime.datetime.now()
cell_stat_sum = arcpy.sa.CellStatistics(in_rasters_or_constants=rasters, 
                                        statistics_type="SUM", 
                                       ignore_nodata="DATA")
time2 = datetime.datetime.now()
runtime1 = time2 - time1
print("Cell statistics method runtime: {0}".format(runtime1))

cell_stat_sum.save(workDir + "cell_stat_sum.tif")

Cell statistics method runtime: 0:00:44.865000


## Example 2 - Sum rasters with raster algebra

Step 1 - List the rasters to sum

In [21]:
arcpy.env.workspace = tinyDir
rasters2 = arcpy.ListRasters()

Step 2 - Sum the rasters <br>
NOTE AGAIN the issue of "nodata" cells comes up again.  The output from the cell below is empty because of nodata cells.  

In [22]:
time3 = datetime.datetime.now()
running_total = arcpy.Raster(rasters2[0])
for raster in rasters2:
    running_total += arcpy.Raster(raster)
time4 = datetime.datetime.now()
runtime2 = time4 - time3
print("Cell statistics method runtime: {0}".format(runtime2))
running_total.save(workDir + "running_total.tif")

Cell statistics method runtime: 0:01:14.188000


To use this method, you first have to fill in the nodatas with zeros.<br>This is why I created the Make01Seasonal function in GAPAnalysis.data https://github.com/nmtarr/GAPAnalysis/blob/master/gapanalysis/data.py, but that also expands the extent to CONUS.  Open source challenge 2 deals with reclassifying rasters, which is a solution to this issue.